In [1]:
import requests
from IPython.display import display, HTML

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, data):
        if not self.head:
            self.head = Node(data)
            return
        current = self.head
        while current.next:
            current = current.next
        current.next = Node(data)

    def display(self):
        current = self.head
        while current:
            print(current.data)
            current = current.next

def fetch_price(platform, symbol):
    """
    Fetches the price of a cryptocurrency from the specified platform.

    Args:
    - platform (str): The platform to fetch the price from.
    - symbol (str): The symbol of the cryptocurrency.

    Returns:
    - float: The price of the cryptocurrency.
    """
    urls = {
        'Coinbase': f'https://api.coinbase.com/v2/prices/{symbol.upper()}-USD/spot',
        'CryptoCompare': f'https://min-api.cryptocompare.com/data/price?fsym={symbol.upper()}&tsyms=USD',
        'Kraken': f'https://api.kraken.com/0/public/Ticker?pair={symbol.upper()}USD',
        'Bitstamp': f'https://www.bitstamp.net/api/v2/ticker/{symbol.lower()}usd/'
        # Add more platforms here if needed
    }

    if platform not in urls:
        print(f"Error: Unsupported platform '{platform}'")
        return 0.0

    url = urls[platform]
    try:
        print(f"Fetching data from {platform} API with URL: {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        if platform == 'Coinbase':
            return float(data['data']['amount'])
        elif platform == 'CryptoCompare':
            return float(data['USD'])
        elif platform == 'Kraken':
            return float(data['result'][list(data['result'].keys())[0]]['c'][0])
        elif platform == 'Bitstamp':
            return float(data['last'])
        # Add more platform-specific parsing logic here if needed
    except requests.RequestException as e:
        print(f"Error: Could not fetch price from {platform} API - {e}")
        return 0.0

def calculate_profit(price_buy, price_sell):
    """
    Calculates the profit based on the price difference between buying and selling.

    Args:
    - price_buy (float): The price at which the cryptocurrency is bought.
    - price_sell (float): The price at which the cryptocurrency is sold.

    Returns:
    - float: The profit based on the price difference.
    """
    return price_sell - price_buy

def fetch_prices_and_calculate_profits():
    """
    Fetches cryptocurrency prices from multiple platforms for various currencies,
    and calculates the profits based on price differences.

    Returns:
    - dict: A dictionary containing the fetched prices and calculated profits.
    """
    platforms = ['Coinbase', 'CryptoCompare', 'Kraken', 'Bitstamp']
    symbols = ['BTC', 'ETH', 'LTC', 'BCH', 'XRP', 'USDC', 'USDT']  # Ensure no duplicate currencies
    
    prices = {platform: {} for platform in platforms}
    for platform in platforms:
        for symbol in symbols:
            prices[platform][symbol] = fetch_price(platform, symbol)

    print(f"Prices fetched: {prices}")

    profits = {symbol: [] for symbol in symbols}
    for symbol in symbols:
        for platform_buy in platforms:
            for platform_sell in platforms:
                if platform_buy != platform_sell:
                    profit = calculate_profit(prices[platform_buy][symbol], prices[platform_sell][symbol])
                    profits[symbol].append({
                        'buy_from': platform_buy,
                        'sell_to': platform_sell,
                        'profit': round(profit, 3)
                    })

    print(f"Profits calculated: {profits}")

    return {
        'prices': prices,
        'profits': profits
    }

def display_best_trading_opportunities(data, symbols):
    """
    Displays the best trading opportunities based on the fetched prices and calculated profits.

    Args:
    - data (dict): A dictionary containing the fetched prices and calculated profits.
    - symbols (list): A list of symbols for cryptocurrencies.
    """
    profits = data['profits']

    best_trades = {symbol: max(profits[symbol], key=lambda x: x['profit']) for symbol in profits}

    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Best Trading Opportunities (Profit per 1 {symbols[0]})</title>
        <style>
            table, th, td {{
                border: 1px solid black;
                border-collapse: collapse;
                padding: 8px;
            }}
        </style>
    </head>
    <body>
        <h1>Best Trading Opportunities (Profit per 1 {symbols[0]})</h1>
        <table>
            <tr>
                <th>Cryptocurrency</th>
                <th>Buy From</th>
                <th>Sell To</th>
                <th>Profit</th>
            </tr>
    """
    for symbol in best_trades:
        html_content += f"""
            <tr>
                <td>{symbol}</td>
                <td>{best_trades[symbol]['buy_from']}</td>
                <td>{best_trades[symbol]['sell_to']}</td>
                <td>{best_trades[symbol]['profit']:.10f}</td>  # Adjust the precision here
            </tr>
        """
    html_content += """
        </table>
    </body>
    </html>
    """

    display(HTML(html_content))

# Fetch prices and calculate profits
data = fetch_prices_and_calculate_profits()

# Display best trading opportunities
display_best_trading_opportunities(data, ['BTC', 'ETH', 'LTC', 'BCH', 'XRP', 'USDC', 'USDT'])


Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/BTC-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/ETH-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/LTC-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/BCH-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/XRP-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/USDC-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/USDT-USD/spot
Fetching data from CryptoCompare API with URL: https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD
Fetching data from CryptoCompare API with URL: https://min-api.cryptocompare.com/data/price?fsym=ETH&tsyms=USD
Fetching data from CryptoCompare API with URL: https://min-api.cryptocompare.com/data/price?fsym=LTC&tsyms=USD
Fetching data from CryptoCompare AP

Cryptocurrency,Buy From,Sell To,Profit
BTC,Coinbase,Bitstamp,18.3350000000
ETH,Kraken,Bitstamp,1.0100000000
LTC,Coinbase,Bitstamp,0.0550000000
BCH,Coinbase,Bitstamp,2.3700000000
XRP,Coinbase,CryptoCompare,-0.0000000000
USDC,Coinbase,CryptoCompare,0.0000000000
USDT,Coinbase,CryptoCompare,-0.0000000000
